In [1]:
import requests
import pandas as pd
import os
import json
from datetime import datetime, timedelta
from dotenv import load_dotenv

#기본 내용 호출

In [2]:
# API키 .env에서 로드
load_dotenv()
API_KEY = os.getenv("SEOUL_API_KEY")

In [3]:
# 요청 url 만들기(테스트용)
SERVICE = "getStnPsgr"
TYPE = "json"
START_INDEX = 1
END_INDEX = 5
target_date = "20260201"
url = f"http://openapi.seoul.go.kr:8088/{API_KEY}/{TYPE}/{SERVICE}/{START_INDEX}/{END_INDEX}/{target_date}"

In [4]:
# 예시로 5개 불러오기
try:
    response = requests.get(url)
    response.raise_for_status()
    data = response.json()

    # url로 확인한 구조: response -> body -> items -> item
    # 데이터 위치 접근
    items = data['response']['body']['items']['item']
        
    # 데이터프레임 변환
    df = pd.DataFrame(items)
    display(df.head())

    # 데이터 타입 확인
    print(df.dtypes)

except Exception as e:
    print(f"통신 에러: {e}")

,pasngDe,pasngHr,lineNm,stnCd,stnNo,stnNm,trnscdSeCd,trnscdSeCdNm,trnscdUserSeCd,trnscdUserSeCdNm,rideNope,gffNope,crtrYmd
0,20260201,23,8호선,2828,821,남위례,2,후불카드,100,우대권,0,0,20260201
1,20260201,23,8호선,2828,821,남위례,2,후불카드,04,청소년,1,3,20260201
2,20260201,23,8호선,2828,821,남위례,2,후불카드,01,일반,18,58,20260201
3,20260201,23,8호선,2828,821,남위례,1,선불카드,99,기타,0,2,20260201
4,20260201,23,8호선,2828,821,남위례,1,선불카드,100,우대권,0,2,20260201


pasngDe             object
pasngHr             object
lineNm              object
stnCd               object
stnNo               object
stnNm               object
trnscdSeCd          object
trnscdSeCdNm        object
trnscdUserSeCd      object
trnscdUserSeCdNm    object
rideNope             int64
gffNope              int64
crtrYmd             object
dtype: object


In [5]:
# 노인/약자, 학생/청소년, 일반, 기타로 그룹 지은 컬럼 생성 시험버전

try:
    response = requests.get(url)
    data = response.json()
    
    items = data['response']['body']['items']['item']
    df = pd.DataFrame(items)
    
    # 숫자형 컬럼 변환 (명세서 기준 int64)
    df['rideNope'] = df['rideNope'].astype(int)
    df['gffNope'] = df['gffNope'].astype(int)


    # 사용자 그룹 함수(get_user_group)
    def get_user_group(code):
        # API에서 코드는 문자열(object)로 들어오므로 문자열로 비교
        if code == '01':
            return '일반'
        elif code in ['02', '03', '04']:
            return '학생/청소년'
        elif code in ['06', '100']:
            return '노인/약자'
        else:
            return '기타' # 그 외 모든 코드 (외국인, 국가유공자 등)

    # 새로운 컬럼 'UserGroup' 생성
    df['UserGroup'] = df['trnscdUserSeCd'].apply(get_user_group)

    
    # 맨 앞에 UserGroup 추가하여 보기 편하게 정렬 및 100개 예시 보기
    cols = ['UserGroup', 'pasngDe', 'pasngHr', 'lineNm', 'stnCd', 'stnNo', 'stnNm', 
            'trnscdSeCd', 'trnscdSeCdNm', 'trnscdUserSeCd', 'trnscdUserSeCdNm', 
            'rideNope', 'gffNope', 'crtrYmd']
    
    final_df = df[cols].head(100)
    display(final_df)

    
    # 그룹별 데이터 분포 확인 (검증용)
    print(final_df['UserGroup'].value_counts())

except Exception as e:
    print(f"에러: {e}")

,UserGroup,pasngDe,pasngHr,lineNm,stnCd,stnNo,stnNm,trnscdSeCd,trnscdSeCdNm,trnscdUserSeCd,trnscdUserSeCdNm,rideNope,gffNope,crtrYmd
0,노인/약자,20260201,23,8호선,2828,821,남위례,2,후불카드,100,우대권,0,0,20260201
1,학생/청소년,20260201,23,8호선,2828,821,남위례,2,후불카드,04,청소년,1,3,20260201
2,일반,20260201,23,8호선,2828,821,남위례,2,후불카드,01,일반,18,58,20260201
3,기타,20260201,23,8호선,2828,821,남위례,1,선불카드,99,기타,0,2,20260201
4,노인/약자,20260201,23,8호선,2828,821,남위례,1,선불카드,100,우대권,0,2,20260201


UserGroup
노인/약자     2
학생/청소년    1
일반        1
기타        1
Name: count, dtype: int64


In [6]:
# 최종 데이터 베이스 적재 1(임포트 및 함수 정의)

# 기본 임포트 내용 
import requests
import pandas as pd
import pymysql
import os
import time
from datetime import datetime, timedelta
from dotenv import load_dotenv

# dbconnect 모듈 임포트
import dbconnect 

# API키 .env에서 로드
load_dotenv()
API_KEY = os.getenv("SEOUL_API_KEY")

# 요청 url 상수
SERVICE = "getStnPsgr"
TYPE = "json"
BATCH_SIZE = 1000 #1000개씩 


## 함수 정의 칸
#db 연결 함수
def get_connection():
    conn = dbconnect.MydbConnect(database='seoul_urban_lab')
    return conn

#사용자 분류 함수
def get_user_group(code):
    if code == '01': return '일반'
    elif code in ['02', '03', '04']: return '어린이/학생/청소년'
    elif code in ['06', '100']: return '노인/약자'
    else: return '기타'

#사용자 분류 함수
def fetch_data(date_str, start_idx, end_idx):
    url = f"http://openapi.seoul.go.kr:8088/{API_KEY}/{TYPE}/{SERVICE}/{start_idx}/{end_idx}/{date_str}"
    try:
        response = requests.get(url, timeout=30)
        response.raise_for_status()
        data = response.json()
        
        if 'getStnPsgr' in data and 'row' in data['getStnPsgr']:
            return data['getStnPsgr']['row']
        elif 'response' in data: 
            return data['response']['body']['items']['item']
        else:
            return []
    except Exception as e:
        print(f"에러: {e}")
        return []
    

def save_to_db(conn, df):
    if len(df) == 0:
        return 0
    
    cursor = conn.cursor()
    
    # 1. 사용할 컬럼 목록을 리스트로 딱 정해둡니다. (순서 중요!)
    columns = [
        'UserGroup', 'pasngDe', 'pasngHr', 'lineNm', 
        'stnCd', 'stnNm', 'trnscdUserSeCd', 'rideNope', 'gffNope'
    ]
    
    # 2. SQL 쿼리 (틀)
    sql = """
    INSERT INTO subway_traffic_log 
    (UserGroup, pasngDe, pasngHr, lineNm, stnCd, stnNm, trnscdUserSeCd, rideNope, gffNope)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
    ON DUPLICATE KEY UPDATE
        rideNope = VALUES(rideNope),
        gffNope = VALUES(gffNope)
    """
    
    # 3. 데이터프레임에서 내가 원하는 컬럼만 뽑아서 사용
    # raw_data는 [[값, 값...], [값, 값...]] 형태
    raw_data = df[columns].values 
    
    count = 0
    
    # 4. 배열을 한 줄씩 꺼내서 루프
    for row in raw_data:
        try:
            # row는 리스트
            # 번호(index)로 하나씩 꺼내서 사용
            # columns 리스트 순서대로 0, 1, 2... 가 매칭
            
            data = (
                row[0],          # UserGroup
                row[1],          # pasngDe
                int(row[2]),     # pasngHr (정수 변환)
                row[3],          # lineNm
                row[4],          # stnCd
                row[5],          # stnNm
                row[6],          # trnscdUserSeCd
                int(row[7]),     # rideNope
                int(row[8])      # gffNope
            )
            
            # 한 줄 실행
            cursor.execute(sql, data)
            count += 1
            
        except Exception as e:
            print(f"에러 발생: {e}")
            # print(f"문제 데이터: {row}")
            
    # 최종 저장
    try:
        conn.commit()
        return count
    except:
        conn.rollback()
        return 0



In [7]:
# 최종 데이터 베이스 적재 2(메인 실행 코드)

# DB 연결 (dbconnect.py)
conn = get_connection()
    
# 날짜 설정 (1월 26일 ~ 2월 1일)
start_date = datetime(2026, 1, 26)
end_date = datetime(2026, 2, 1)
    
current_date = start_date
total_inserted = 0 # 전체 갯수 세는 공간
    
while current_date <= end_date:

    target_date_str = current_date.strftime("%Y%m%d")
    print(f"\n Date: {target_date_str} 데이터 수집 시작")
        
    start_idx = 1
    daily_total = 0
        
    while True:
        end_idx = start_idx + BATCH_SIZE - 1
        #배치 사이즈 1000
        print(f"{start_idx} ~ {end_idx}", end="\r")
            
        # 추출 함수 사용
        items = fetch_data(target_date_str, start_idx, end_idx)
            
        if not items:
            print(f"데이터 끝 (Page End)")
            break
                
        # 그룹 함수 사용
        df = pd.DataFrame(items)
        df['UserGroup'] = df['trnscdUserSeCd'].apply(get_user_group)
            
        # 저장 함수 사용
        count = save_to_db(conn, df)
        daily_total += count
            
        if len(items) < BATCH_SIZE:
            break

        #1000개 작업하고 시작 인덱스 1000 더하고 30초 쉬기    
        start_idx += BATCH_SIZE
        time.sleep(0.2)
            
    print(f"{target_date_str} 완료! 저장 건수: {daily_total}")

    total_inserted += daily_total
    current_date += timedelta(days=1)
        
conn.close()
print("\n" + "="*50)
print(f"전체 작업 완료! 총 {total_inserted}건 처리됨.")
print("="*50)


데이터베이서 연결 시작!
MySQL DB 연결 성공

 Date: 20260126 데이터 수집 시작
20260126 완료! 저장 건수: 63090

 Date: 20260127 데이터 수집 시작
20260127 완료! 저장 건수: 63589

 Date: 20260128 데이터 수집 시작
20260128 완료! 저장 건수: 63636

 Date: 20260129 데이터 수집 시작
20260129 완료! 저장 건수: 63927

 Date: 20260130 데이터 수집 시작
20260130 완료! 저장 건수: 64828

 Date: 20260131 데이터 수집 시작
20260131 완료! 저장 건수: 64489

 Date: 20260201 데이터 수집 시작
20260201 완료! 저장 건수: 61567

전체 작업 완료! 총 445126건 처리됨.
